<a href="https://colab.research.google.com/github/davidetassinari/web-scraper-analyzer/blob/main/20250223_Async_scraper_Good_enough_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents

1. Introduction
1. Mount Your Google Drive
1. Load the Input Website List in `.csv` Form
1. Perform a Simple Crawl of the Websites in the List
1. Scrape Plain Text from the List of Crawled Webpages for Each Site
1. Analyse Website Text Using OpenAI's Async API
1. Clean-up and Parsing of LLM Output
1. Organise All Information in a Spreadsheet


## 1. Introduction

Install any needed dependencies.

In [ ]:
!pip install openai requests pandas beautifulsoup4

## 2. Mount Your Google Drive

Mount your Drive in order to access your `.csv` file.

You need to do this every time you open this notebook or reconnect to an environment.

Run the next cell to authorise access:
- Select "Connect to Google Drive" in the small popup
- Choose the correct account from the list in the popup window
- Sign in to that account
- The first time you authorise accesso to Drive you will need to select all permissions. After that you can just select "Continue" unless you choose to manually review the permissions.

After you run this code block you should see the message `Mounted at /content/drive` at the bottom.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Load the Input Website List in `.csv` Form

Place your `.csv` file (e.g., `venture_websites.csv`) in a known folder on your
Drive.

Note the file path, as you will use it to load your data.

The `.csv` file must have a header row with the label "Website".

In [ ]:
import pandas as pd

# Replace the file path below with the actual path to your CSV file in Drive
csv_file_path = '/content/drive/MyDrive/Coding/VentureBuilders/0_website_csv/0_venture_websites.csv'

df = pd.read_csv(csv_file_path)

print("CSV loaded. Number of records:", len(df))

## 4. Perform a Simple Crawl of the Websites in the List

The following code will loop through each URL in your `.csv` file, performing a simple two-level deep crawl and deleting duplicates.

Websites that are unreachable or return errors are simply skipped.

The results are saved into a new `JSON` file in a specified folder.

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd # Might be legacy from when output was saved as csv
import concurrent.futures
import json

pages_to_scrape = '/content/drive/MyDrive/Coding/VentureBuilders/1_web_page_lists/1_website_crawl.json'

def get_links_from_page(url):
    """Scrape all internal links from a given webpage."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        base_domain = urlparse(url).netloc

        links = set()
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            absolute_url = urljoin(url, href)
            if urlparse(absolute_url).netloc == base_domain:
                links.add(absolute_url)

        return links

    except Exception as e:
        print(f"Failed to retrieve {url}: {e}")
        return set()


def normalize_url(url):
    """Normalize URL by removing fragments and trailing slashes."""
    parsed = urlparse(url)._replace(fragment='')
    normalized_url = parsed.geturl().rstrip('/')
    return normalized_url


def check_url(url):
    """Check if a URL is reachable."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return True
    except Exception as e:
        print(f"URL check failed for {url}: {e}")
        return False


def scrape_website_links(input_csv, output_json, max_depth=2):
    """Scrapes websites to a given depth and writes to a JSON file."""
    websites = pd.read_csv(input_csv)
    result_data = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for _, row in websites.iterrows():
            base_url = normalize_url(row['Website'])

            # Check if the base URL is reachable
            if not check_url(base_url):
                print(f"Skipping unreachable URL: {base_url}")
                continue

            visited = set([base_url])
            all_pages = {base_url}
            future_to_url = {executor.submit(get_links_from_page, base_url): (base_url, 1)}

            while future_to_url:
                completed, _ = concurrent.futures.wait(future_to_url.keys(), return_when=concurrent.futures.FIRST_COMPLETED)
                for future in completed:
                    url, depth = future_to_url.pop(future)
                    try:
                        links = future.result()
                        new_links = {normalize_url(link) for link in links if normalize_url(link) not in visited}
                        all_pages.update(new_links)
                        visited.update(new_links)

                        if depth < max_depth:
                            for link in new_links:
                                future_to_url[executor.submit(get_links_from_page, link)] = (link, depth + 1)
                    except Exception as e:
                        print(f"Error processing {url}: {e}")

            result_data.append({
                "base_website": base_url,
                "pages": list(all_pages)
            })

    # Save to JSON file
    with open(output_json, 'w') as f:
        json.dump(result_data, f, indent=4)
    print(f"Scrape completed and saved to {output_json}")


# Example usage:
scrape_website_links(csv_file_path, pages_to_scrape)

## 5. Scrape Plain Text from the List of Crawled Webpages for Each Site

Once all existing websites have been crawled, all the pages are visited sequentially and scraped for plain text. `PDF` files are skipped.

Text length is limited to `10000` characters for each page to avoid feeding excessively large files to the LLM in the following step.

One output file is created for each website in the original list, with a simple `JSON` schema. The filename is base on the original URL in the list.

The original URL is preserved in each of the following steps for compatibility with the initial website list.

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Paths and directories
input_path = pages_to_scrape
# input_path = '/content/drive/MyDrive/Coding/VentureBuilders/1_web_page_lists/1_website_crawl.json'

output_dir = '/content/drive/MyDrive/Coding/VentureBuilders/2_full_site_scrapes'

os.makedirs(output_dir, exist_ok=True)

# Load the intermediate JSON file with base URLs
with open(input_path, 'r') as f:
    base_urls = json.load(f)

def scrape_page_content(url):
    """Scrape text content from a single webpage."""
    try:
        if url.lower().endswith('.pdf'):
            return {"url": url, "error": "Skipped PDF file"}

        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            return {"url": url, "error": f"HTTP {response.status_code}"}

        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else "No title"
        text = soup.get_text(separator=' ', strip=True)

        return {
            "url": url,
            "title": title,
            "content": text[:10000]  # limit text length to avoid excessive data
        }

    except Exception as e:
        return {"url": url, "error": str(e)}

def sanitize_filename(url):
    """Create a legible filename by replacing URL dots with underscores."""
    return url.replace('https://', '').replace('http://', '').replace('www.', '').replace('/', '_').replace('.', '_')

def is_url_accessible(url):
    """Check if the base URL is accessible."""
    try:
        response = requests.get(url, timeout=10)
        return response.status_code == 200
    except Exception as e:
        print(f"Failed to connect to {url}: {e}")
        return False

# Iterate over each base URL and its pages
for entry in base_urls:
    base_url = entry['base_website']
    site_name = sanitize_filename(base_url)
    print(f"Checking site: {base_url}")

    # Check if the site is accessible
    if not is_url_accessible(base_url):
        print(f"Skipping site: {base_url} due to connection issues")
        continue

    site_content = [{
        "original_url": base_url,
        "sanitized_name": site_name
    }]

    for page_url in entry['pages']:
        if page_url.lower().endswith('.pdf'):
            print(f"Skipping PDF file: {page_url}")
            continue
        print(f"Scraping page: {page_url}")
        page_data = scrape_page_content(page_url)
        site_content.append(page_data)

    # Save the scraped content to a JSON file
    output_file = os.path.join(output_dir, f"scraped_{site_name}.json")
    with open(output_file, 'w') as f:
        json.dump(site_content, f, indent=4)

    print(f"Finished scraping {base_url}. Data saved to: {output_file}")

print("Scraping complete for all sites.")

## 6. Analyse Website Text Using OpenAI's Async API

Once all scraped text files have been created in the chosen folder, each file is sent for analysis to OpenAI's Async API.

The chosen model is currently (as of 2025-02-23) `GPT-4o-mini` since:

- It is adequate for simple text analysis and output
- It allows for the largest async requests (200k tokens)
- It has the largest number of async requests (500 requests per minute)
- It has the largest daily limit (2 M tokens per day)

The API key is stored as a Colab Secret as `OPENAI_API_KEY`.

The prompt needs to be customised to perform the required analysis.

The prompt also specifies the `JSON` schema of the output as a quick and dirty stand-in for a structured output. The following code block performs some cleanup and some local parsing to ensure consistency in `JSON` keys.

For each input scraped text `JSON` file in the folder, an output analysis `JSON` file is created in the specified folder. File names are maintained consistent with a prefix that clarifies their stage for clarity.

Async calls allow for some flexibility when the number of tokens per minute is close to the limit.

In [ ]:
import os
import json
import asyncio
from openai import AsyncOpenAI
from google.colab import userdata

# Paths and directories
# output_dir = '/content/drive/MyDrive/Coding/VentureBuilders/Full_Site_Scrapes'
output_dir = '/content/drive/MyDrive/Coding/VentureBuilders/2_full_site_scrapes' # output_dir repeated so I can only run this code block
analysis_dir = '/content/drive/MyDrive/Coding/VentureBuilders/3_analysis_results'
os.makedirs(analysis_dir, exist_ok=True)

# Initialize OpenAI client

API_KEY = userdata.get('OPENAI_API_KEY')
# print(API_KEY)
client = AsyncOpenAI(api_key=API_KEY)

async def analyze_site_content(file_path, output_file_path):
    """Analyze the content of a site using the OpenAI API."""
    with open(file_path, 'r') as f:
        site_data = json.load(f)

    # Initialize original_url and determine where the page data is stored.
    original_url = None
    if isinstance(site_data, dict) and "original_url" in site_data:
        original_url = site_data["original_url"]
        pages = site_data.get("pages", [])
    elif isinstance(site_data, list) and len(site_data) > 0 and isinstance(site_data[0], dict) and "original_url" in site_data[0]:
        original_url = site_data[0]["original_url"]
        pages = site_data[1:]  # assume the first element is metadata and the rest are pages
    else:
        pages = site_data  # assume site_data is a list of pages if no original_url is present

    # Construct a text block from the pages for the prompt.
    content_text = "\n\n".join(
        f"Title: {page.get('title', 'No title')}\nContent: {page.get('content', '')[:3000]}" # Not sure this 3000 limit makes sense. Tokens? Characters?
        for page in pages
    )

    prompt = (
        "You are a skilled MBA graduate with experience in the startup and venture building world. "
        "Analyse the following information and summarise it based on the following attributes:\n\n"
        "## Venture Builder Name\n"
        "The name of the Venture Builder organization.\n\n"
        "## Guild\n"
        "The Guild is the infrastructure package that Venture builders supply to their startup companies.\n"
        "It includes the entrepreneurs-in-residence who will form the backbone of the new ventures, technological know-how, financing, and physical spaces.\n"
        "The Guild may be shared horizontally between different startups in the portfolio to increase cost-effectiveness, or it can be distributed vertically for each business.\n\n"
        "## Control\n"
        "Venture builders have different rules governing the equity they hold in startups, according to their coaching intensity and required degree of control.\n\n"
        "## Focus\n"
        "Venture builders can choose to operate across different sectors, covering a wide spectrum of technologies and markets, or they may operate vertically in a specific sector or on a specific technological solution (e.g. Artificial Intelligence or Deep Tech). "
        "The first approach is called the Generalist while the second is called Specialist.\n\n"
        "## Idea\n"
        "Idea generation is a critical component of the Business Model of venture builders. Sourcing might be done externally or internally. "
        "External sourcing may take two forms: fostering external early-stage firms or scanning the market for the best venture to produce a Copycat.\n\n"
        "## Volume\n"
        "Startup studios can develop a few or many businesses concurrently.\n\n"
        "## Funding\n"
        "Startup studios can choose to finance their portfolio with their own resources or can choose external funding. "
        "Hybrid solutions are common, with external partners piggybacking onto the funding that the Venture-Builder makes available for the start-ups.\n\n"
        "## Scaling\n"
        "Startup studios are concerned with the timing and strategy of their exit with regards to scaling. Relevant terminology includes, but is not limited to, "
        "scaling, scalers, scaleup, unicors and gazelles. The main focus of this section is on at what time in the startup life the startup studio aims to exit.\n"
        "## Final remarks\n"
        "Any final conclusions.\n\n"
        "Instructions:\n"
        "* Your analysis should be thorough, comprehensive and objective.\n"
        "* Rely exclusively on information found on the official website of the venture builder.\n"
        "* Avoid assumptions or external sources.\n"
        "* If any attribute information is not available on the website, indicate it as not disclosed or not available, by simply using \"Not disclosed\" or \"Not available\" without additional detail.\n\n"
        "* The output should be formatted according to JSON standards for ease of parsing, with the category as a key and the analysis content as the value.\n"
        "* Each JSON value for the main sections should be in natural language format for ease of consultation, and not contain any nested dicts or key-value pairs.\n\n"
        "* The size of the output value for each category and each venture builder should be commensurate to the amount of information retrieved and may be lengthy if required.\n"
        "The text to be analyzed follows.\n\n\n" + content_text
    )

    response = await client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gpt-4o-mini",
    )

    analysis = response.choices[0].message.content.strip()

    # Build output JSON with the preserved original_url (if available) at the beginning.
    output_data = {}
    if original_url is not None:
        output_data["original_url"] = original_url
    output_data["analysis"] = analysis

    # Save analysis to the provided output file path.
    with open(output_file_path, 'w') as f:
        json.dump(output_data, f, indent=4)

    print(f"Analysis saved to {output_file_path}")

async def main():
    """Run analyses for all JSON files in the output directory."""
    tasks = []
    output_dir_real = os.path.realpath(output_dir)
    for file_name in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file_name)
        if os.path.isdir(file_path):
            continue
        if os.path.realpath(os.path.dirname(file_path)) != output_dir_real:
            continue
        if file_name.endswith('.json'):
            # Create new filename by replacing "scraped_" with "analyzed_"
            if file_name.startswith("scraped_"):
                analyzed_file_name = "analyzed_" + file_name[len("scraped_"):]
            else:
                analyzed_file_name = "analyzed_" + file_name
            output_file_path = os.path.join(analysis_dir, analyzed_file_name)
            tasks.append(analyze_site_content(file_path, output_file_path))
    await asyncio.gather(*tasks)


import nest_asyncio
nest_asyncio.apply()
asyncio.run(main())
print("Batch analysis complete.")

Analysis saved to /content/drive/MyDrive/Coding/VentureBuilders/3_analysis_results/analyzed_sap_io.json
Batch analysis complete.


## 7. Clean-up and Parsing of LLM Output

Since LLM outputs do not strictly adhere to a `JSON` schema (unless structured output is implemented), a cleanup and parsing step is required.

Any further internal `JSON` structure in the analysis values (further lists or dicts) is flattened to improve legibility in the final spreadsheet.

Again, for each file in the analysis result folder a new parsed file is created in the specified folder and the URL from the initial `.csv` is preserved.

In [ ]:
import os
import json
import re

# Paths and directories
analysis_dir = '/content/drive/MyDrive/Coding/VentureBuilders/3_analysis_results'
parsed_dir = '/content/drive/MyDrive/Coding/VentureBuilders/4_parsed_results'
os.makedirs(parsed_dir, exist_ok=True)

def normalize_key(key):
    """Convert a key to lowercase and remove spaces, dashes, and underscores."""
    return key.lower().replace(' ', '').replace('-', '').replace('_', '')

def parse_analysis_file(file_path):
    """Parse the analysis text into structured JSON with flattened natural language output,
       preserving the original URL and normalizing keys."""
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Capture the original URL from the input JSON
    original_url = data.get('original_url', '')

    analysis_text = data.get('analysis', '')

    # Extract JSON content from the code block
    json_match = re.search(r'```json\n(.*?)\n```', analysis_text, re.DOTALL)
    if json_match:
        clean_json = json_match.group(1)
        try:
            parsed_data = json.loads(clean_json)
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for {file_path}")
            parsed_data = {}
    else:
        print(f"No valid JSON found in {file_path}")
        parsed_data = {}

    # Flatten the structure into plain text paragraphs with "Key: value." formatting
    flattened_data = {}
    for key, value in parsed_data.items():
        norm_key = normalize_key(key)
        if isinstance(value, dict):
            # Build a string for each normalized subkey/value pair in the dict
            inner_text = ". ".join(f"{normalize_key(subkey)}: {subvalue}" for subkey, subvalue in value.items())
            flattened_data[norm_key] = inner_text
        elif isinstance(value, list):
            # Join list items as comma-separated text
            list_text = ", ".join(str(item) for item in value)
            flattened_data[norm_key] = list_text
        else:
            flattened_data[norm_key] = value

    # Remove any square brackets that might appear in the text
    flattened_data = {k: v.replace("[", "").replace("]", "") for k, v in flattened_data.items()}

    # Add the original URL with normalized key
    flattened_data["originalurl"] = original_url

    # Determine the output file name by replacing "analyzed_" with "parsed_"
    file_basename = os.path.basename(file_path)
    if file_basename.startswith("analyzed_"):
        parsed_file_name = "parsed_" + file_basename[len("analyzed_"):]
    else:
        parsed_file_name = "parsed_" + file_basename

    # Save the flattened data to a new JSON file in parsed_dir
    output_file = os.path.join(parsed_dir, parsed_file_name)
    with open(output_file, 'w') as f:
        json.dump(flattened_data, f, indent=4)

    print(f"Parsed analysis saved to {output_file}")

# Process all analysis files
for file_name in os.listdir(analysis_dir):
    if file_name.endswith('.json'):
        file_path = os.path.join(analysis_dir, file_name)
        parse_analysis_file(file_path)

print("Parsing complete for all analysis files.")


Parsed analysis saved to /content/drive/MyDrive/Coding/VentureBuilders/4_parsed_results/parsed_sap_io.json
Parsing complete for all analysis files.


## 8. Organise All Information in a Spreadsheet

The parsed analysis output is finally inserted in a Google Sheets spreadsheet.

Column headers correspond to the analysis requested in the LLM prompt.

In [ ]:
import os
import json
import gspread
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build

# Authenticate and initialize gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Paths and directories
parsed_dir = '/content/drive/MyDrive/Coding/VentureBuilders/4_parsed_results'
sheet_name = 'venture_builders_analysis'
project_root_name = 'VentureBuilders'  # This folder corresponds to '/content/drive/MyDrive/Coding/VentureBuilders/'

# Open or create the Google Sheet
try:
    sh = gc.open(sheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create(sheet_name)

# Use Google Drive API to move the spreadsheet into the project root folder
drive_service = build('drive', 'v3', credentials=creds)

# Search for the folder "VentureBuilders" in MyDrive
results = drive_service.files().list(
    q=f"mimeType='application/vnd.google-apps.folder' and name='{project_root_name}' and trashed=false",
    fields="files(id, name)"
).execute()
items = results.get('files', [])
if items:
    project_folder_id = items[0]['id']
else:
    # Create the folder if it doesn't exist
    file_metadata = {
        'name': project_root_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = drive_service.files().create(body=file_metadata, fields='id').execute()
    project_folder_id = folder.get('id')

# Move the spreadsheet to the project folder
file_id = sh.id
file_info = drive_service.files().get(fileId=file_id, fields='parents').execute()
previous_parents = ",".join(file_info.get('parents'))
drive_service.files().update(
    fileId=file_id,
    addParents=project_folder_id,
    removeParents=previous_parents,
    fields='id, parents'
).execute()

# Select the "Analysis" worksheet or create it if it doesn't exist
try:
    worksheet = sh.worksheet('Analysis')
except gspread.exceptions.WorksheetNotFound:
    worksheet = sh.add_worksheet(title='Analysis', rows='1000', cols='10')

# Define headers for the data
headers = [
    "Original URL", "Venture Builder Name", "Guild", "Control",
    "Focus", "Idea", "Volume", "Funding", "Scaling", "Final remarks"
]

# Ensure the header row is present by updating cells A1:J1 using named arguments
worksheet.update(values=[headers], range_name='A1:J1')

# Read JSON files from parsed_dir and append each as a new row
for file_name in os.listdir(parsed_dir):
    if file_name.endswith('.json'):
        file_path = os.path.join(parsed_dir, file_name)
        with open(file_path, 'r') as f:
            parsed_data = json.load(f)

        row = [
            parsed_data.get("originalurl", ""),
            parsed_data.get("venturebuildername", ""),
            parsed_data.get("guild", ""),
            parsed_data.get("control", ""),
            parsed_data.get("focus", ""),
            parsed_data.get("idea", ""),
            parsed_data.get("volume", ""),
            parsed_data.get("funding", ""),
            parsed_data.get("scaling", ""),
            parsed_data.get("finalremarks", "")
        ]
        worksheet.append_row(row, value_input_option='USER_ENTERED')
        print(f"Analysis appended for {file_name}")

print("All parsed analysis files have been appended into the Google Sheet.")


Analysis appended for parsed_sap_io.json
All parsed analysis files have been appended into the Google Sheet.
